In [1]:
import pandas as pd
import numpy as np

# 計算Call Put Volatility Spread，放在df_cv裡

In [2]:
#讀入excel檔，設定date的格式為string
df1 = pd.read_excel('/Users/Kang/Desktop/389703_index ETF option implied volatility.xlsx', dtype={'date': str})
df1

,secid,date,days,strike_price,impl_volatility,delta,cp_flag,ticker,index_flag
0,106422,20100104,30,13.342624,0.258635,0.514782,C,EWT,0
1,106422,20100104,30,13.342624,0.255357,-0.485584,P,EWT,0
2,106422,20100105,30,13.292614,0.223508,0.512775,C,EWT,0
3,106422,20100105,30,13.292614,0.248231,-0.485996,P,EWT,0
4,106422,20100106,30,13.372617,0.239470,0.513687,C,EWT,0
5,106422,20100106,30,13.372617,0.224426,-0.487377,P,EWT,0
6,106422,20100107,30,13.272582,0.250167,0.514298,C,EWT,0
7,106422,20100107,30,13.272582,0.235154,-0.486752,P,EWT,0
8,106422,20100108,30,13.342599,0.241869,0.513824,C,EWT,0
9,106422,20100108,30,13.342599,0.252507,-0.485747,P,EWT,0


In [3]:
#建立一個調整用的implied volatility欄，不將原本欄位覆蓋掉
#在該調整欄中，將原本cp_flag欄裡標示為P者，impl_volatility之數值乘(-1)，利於等一下直接相加
df1['impl_vol_adj'] = np.nan
df1['impl_vol_adj'] = [df1['impl_volatility'][i] * (-1) if df1['cp_flag'][i] == 'P' else df1['impl_volatility'][i]  for i in range(len(df1))]

In [4]:
#根據ticker, date分群，每群中都有一個C一個P，因為剛才將P乘了(-1)，現在直接相加
#轉成字典，其中key就是ticker跟date
group = df1.groupby(['ticker', 'date'])['impl_vol_adj'].sum()
my_dict = group.to_dict()
my = list(my_dict.keys())

In [5]:
#看全部有多少不重複的ticker, date，將之建成list
all_ticker = df1['ticker'].unique().tolist()
all_day = df1['date'].unique().tolist()

#新增一個dataframe，index是所有date，column是所有ticker，用來儲存C-P
df_cv = pd.DataFrame(index = all_day, columns = all_ticker)

#一個個檢查剛才的字典，將value對應ticker, date，丟到df_cv裡正確的格子
for i in range(len(my)):
    df_cv[my[i][0]][my[i][1]] = my_dict[my[i]]

#將date拆成可被datetime辨識的格式，並當成新的index
all_day_dt = [all_day[i][:4] + '-' + all_day[i][4:6] + '-' + all_day[i][6:] for i in range(len(all_day))]
df_cv = df_cv.set_index(pd.to_datetime(all_day_dt))
df_cv

,EWT,IVV,TLT,AGG,VNQ,VWO,VPL,VGK,PFF,JNK,EMB
2010-01-04,0.003278,-0.022808,-0.011812,-0.008343,-0.017404,-0.029856,-0.105985,-0.020047,-0.018722,-0.028709,NaN
2010-01-05,-0.024723,-0.02111,-0.011864,-0.015141,-0.043329,-0.025098,-0.136337,-0.018335,-0.039389,-0.042018,NaN
2010-01-06,0.015044,-0.002362,-0.00123,-0.015733,-0.01357,-0.012875,-0.122278,-0.03876,-0.011795,-0.020093,NaN
2010-01-07,0.015013,-0.021688,-0.016862,-0.010401,-0.01335,-0.008471,-0.136692,-0.012247,-0.027533,-0.021328,NaN
2010-01-08,-0.010638,-0.034564,-0.019699,-0.005248,-0.024479,-0.025989,-0.130503,-0.02392,-0.014881,-0.02096,NaN
2010-01-11,0.002651,-0.019375,-0.004507,-0.00997,-0.019434,-0.021469,-0.122861,-0.035444,-0.019489,-0.005621,NaN
2010-01-12,-0.011017,-0.006778,-0.009392,-0.009819,-0.005587,-0.024127,-0.103705,-0.024209,-0.04955,-0.037238,NaN
2010-01-13,0.010279,-0.049444,-0.000495,-0.009606,-0.035015,-0.018444,-0.117495,-0.037258,-0.030415,-0.007193,NaN
2010-01-14,0.01998,-0.0182,-0.018479,-0.003176,-0.014995,-0.010062,-0.104326,-0.006119,-0.039678,-0.017933,NaN
2010-01-15,0.001169,-0.001841,-0.011269,-0.013783,-7.4e-05,0.00292,-0.127787,0.01375,-0.037876,-0.007998,NaN


In [6]:
#df_cv.to_csv('/Users/Kang/Desktop/CVol-PVol.csv')

# 計算Call Put average volatility，放在df_iv裡

In [7]:
#The implied volatility (IVol) of an individual stock is calculated as the average of the call and put implied volatilities
group_1 = df1.groupby(['ticker', 'date'])['impl_volatility'].mean()
my_dict_1 = group_1.to_dict()
my_1 = list(my_dict_1.keys())

#儲存IVol
df_iv = pd.DataFrame(index = all_day, columns = all_ticker)

#一個個檢查剛才的字典，將value對應ticker, date，丟到df_iv裡正確的格子
for i in range(len(my_1)):
    df_iv[my_1[i][0]][my_1[i][1]] = my_dict_1[my_1[i]]

In [8]:
df_iv

,EWT,IVV,TLT,AGG,VNQ,VWO,VPL,VGK,PFF,JNK,EMB
20100104,0.256996,0.185248,0.139929,0.0581435,0.306209,0.279102,0.277116,0.225656,0.081134,0.0962375,NaN
20100105,0.23587,0.173754,0.13268,0.0559105,0.304941,0.272636,0.299945,0.227878,0.0718085,0.093039,NaN
20100106,0.231948,0.169038,0.137968,0.0557075,0.30877,0.276535,0.297321,0.221963,0.0783815,0.0843015,NaN
20100107,0.242661,0.166406,0.136298,0.0554885,0.297902,0.273164,0.300565,0.228272,0.0758985,0.080205,NaN
20100108,0.247188,0.158388,0.131053,0.056333,0.279492,0.270031,0.294262,0.221641,0.0724055,0.080863,NaN
20100111,0.239488,0.16276,0.135248,0.051202,0.276202,0.266026,0.29976,0.218525,0.0772585,0.0825705,NaN
20100112,0.240556,0.169061,0.125477,0.0562425,0.279145,0.270022,0.290826,0.226621,0.072558,0.077161,NaN
20100113,0.236663,0.170055,0.126487,0.056928,0.270678,0.25969,0.289932,0.219701,0.0903145,0.0958755,NaN
20100114,0.224588,0.155704,0.123159,0.056181,0.269339,0.256274,0.28402,0.216224,0.073796,0.0822945,NaN
20100115,0.217469,0.161907,0.116676,0.0558615,0.263707,0.254833,0.286531,0.223465,0.069395,0.094532,NaN


# 整理df2的ETF realized volatility，放在df_realized裡

In [9]:
df2 = pd.read_excel('/Users/Kang/Desktop/389703_index ETF realized volatility.xlsx', dtype={'date': str})
df2

,secid,date,days,volatility,ticker,index_flag
0,106422,20100104,30,0.151229,EWT,0
1,106422,20100105,30,0.150209,EWT,0
2,106422,20100106,30,0.149950,EWT,0
3,106422,20100107,30,0.151017,EWT,0
4,106422,20100108,30,0.150081,EWT,0
5,106422,20100111,30,0.146572,EWT,0
6,106422,20100112,30,0.155852,EWT,0
7,106422,20100113,30,0.155840,EWT,0
8,106422,20100114,30,0.153183,EWT,0
9,106422,20100115,30,0.149721,EWT,0


In [10]:
#分群後直接列出，將ticker裡的etf變成column，轉換資料格式
group_2 = df2.groupby(['ticker', 'date'])['volatility'].apply(float)
my_dict_2 = group_2.to_dict()
my_2 = list(my_dict_2.keys())

df_realized = pd.DataFrame(index = all_day, columns = all_ticker)

#一個個檢查剛才的字典，將value對應ticker, date，丟到df_realized裡正確的格子
for i in range(len(my_2)):
    df_realized[my_2[i][0]][my_2[i][1]] = my_dict_2[my_2[i]]

In [11]:
df_realized

,EWT,IVV,TLT,AGG,VNQ,VWO,VPL,VGK,PFF,JNK,EMB
20100104,0.151229,0.113223,0.11722,0.036221,0.186989,0.190307,0.153343,0.206703,0.035631,0.070154,NaN
20100105,0.150209,0.110341,0.117819,0.039711,0.182152,0.185299,0.149775,0.201322,0.035839,0.073211,NaN
20100106,0.14995,0.109823,0.124757,0.039414,0.162974,0.181976,0.14533,0.201734,0.035494,0.07319,NaN
20100107,0.151017,0.100352,0.12483,0.038925,0.1633,0.175166,0.146871,0.181166,0.035319,0.072993,NaN
20100108,0.150081,0.1004,0.124621,0.039079,0.166386,0.175094,0.149711,0.18261,0.03667,0.073095,NaN
20100111,0.146572,0.101945,0.123365,0.039473,0.161956,0.179765,0.153562,0.187742,0.038993,0.076189,NaN
20100112,0.155852,0.106926,0.137202,0.04523,0.171544,0.185575,0.150767,0.192814,0.038584,0.076181,NaN
20100113,0.15584,0.107701,0.142259,0.047527,0.174275,0.185372,0.151104,0.195433,0.038441,0.07246,NaN
20100114,0.153183,0.105186,0.151708,0.047607,0.16937,0.18372,0.150127,0.191496,0.0388,0.071228,NaN
20100115,0.149721,0.115139,0.153514,0.047637,0.171206,0.190271,0.152,0.203892,0.040269,0.074133,NaN


# df_realized - df_iv，得到RVol - IVol，存在df_ri裡

In [12]:
df_ri = pd.DataFrame(index = all_day, columns = all_ticker)

In [13]:
#將兩張表直接相減，過程比較久，可以改良流程
for i in all_ticker:
    for j in all_day:
        df_ri[i][j] = df_realized[i][j] - df_iv[i][j]

In [14]:
#將date拆成可被datetime辨識的格式，並當成新的index
all_day_dt = [all_day[i][:4] + '-' + all_day[i][4:6] + '-' + all_day[i][6:] for i in range(len(all_day))]
df_ri = df_ri.set_index(pd.to_datetime(all_day_dt))
df_ri

,EWT,IVV,TLT,AGG,VNQ,VWO,VPL,VGK,PFF,JNK,EMB
2010-01-04,-0.105767,-0.072025,-0.022709,-0.0219225,-0.11922,-0.088795,-0.123773,-0.0189525,-0.045503,-0.0260835,NaN
2010-01-05,-0.0856605,-0.063413,-0.014861,-0.0161995,-0.122789,-0.087337,-0.15017,-0.0265555,-0.0359695,-0.019828,NaN
2010-01-06,-0.081998,-0.059215,-0.013211,-0.0162935,-0.145796,-0.0945585,-0.151991,-0.020229,-0.0428875,-0.0111115,NaN
2010-01-07,-0.0916435,-0.066054,-0.011468,-0.0165635,-0.134602,-0.0979975,-0.153694,-0.0471065,-0.0405795,-0.007212,NaN
2010-01-08,-0.097107,-0.057988,-0.0064325,-0.017254,-0.113105,-0.0949375,-0.14455,-0.039031,-0.0357355,-0.007768,NaN
2010-01-11,-0.0929165,-0.0608155,-0.0118835,-0.011729,-0.114246,-0.0862605,-0.146198,-0.030783,-0.0382655,-0.0063815,NaN
2010-01-12,-0.0847035,-0.062135,0.011725,-0.0110125,-0.107601,-0.0844465,-0.140059,-0.0338065,-0.033974,-0.00098,NaN
2010-01-13,-0.0808235,-0.062354,0.0157715,-0.009401,-0.0964035,-0.074318,-0.138828,-0.024268,-0.0518735,-0.0234155,NaN
2010-01-14,-0.071405,-0.050518,0.0285485,-0.008574,-0.0999685,-0.072554,-0.133893,-0.0247275,-0.034996,-0.0110665,NaN
2010-01-15,-0.0677485,-0.0467685,0.0368385,-0.0082245,-0.092501,-0.064562,-0.134531,-0.019573,-0.029126,-0.020399,NaN


In [15]:
#df_ri.to_csv('/Users/Kang/Desktop/RVol-IVol.csv')